In [89]:
from scipy.io import loadmat
import pandas as pd
import numpy as np
import os

In [90]:
CWD = "home/jupyter/aiforsea/"

In [2]:
x = loadmat('cars_meta.mat')
type(x)

dict

In [3]:
x

{'__globals__': [],
 '__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sat Dec 14 14:13:07 2013',
 '__version__': '1.0',
 'class_names': array([[array(['AM General Hummer SUV 2000'], dtype='<U26'),
         array(['Acura RL Sedan 2012'], dtype='<U19'),
         array(['Acura TL Sedan 2012'], dtype='<U19'),
         array(['Acura TL Type-S 2008'], dtype='<U20'),
         array(['Acura TSX Sedan 2012'], dtype='<U20'),
         array(['Acura Integra Type R 2001'], dtype='<U25'),
         array(['Acura ZDX Hatchback 2012'], dtype='<U24'),
         array(['Aston Martin V8 Vantage Convertible 2012'], dtype='<U40'),
         array(['Aston Martin V8 Vantage Coupe 2012'], dtype='<U34'),
         array(['Aston Martin Virage Convertible 2012'], dtype='<U36'),
         array(['Aston Martin Virage Coupe 2012'], dtype='<U30'),
         array(['Audi RS 4 Convertible 2008'], dtype='<U26'),
         array(['Audi A5 Coupe 2012'], dtype='<U18'),
         array(['Audi TTS Coupe 2012'], d

In [5]:
classes = x["class_names"]

In [6]:
classes.shape

(1, 196)

In [11]:
df = pd.DataFrame(classes.T, columns=["labels"])

In [14]:
df.labels = df.labels.apply(lambda x: x[0])

In [15]:
df.head()

,labels
0,AM General Hummer SUV 2000
1,Acura RL Sedan 2012
2,Acura TL Sedan 2012
3,Acura TL Type-S 2008
4,Acura TSX Sedan 2012


In [17]:
df.to_csv("cars_meta_processed.csv", columns=["labels"], index_label="index")

# Explore the data

In [19]:
x = loadmat("cars_train_annos.mat")
type(x)

dict

In [20]:
x

{'__globals__': [],
 '__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sat Dec 14 14:13:07 2013',
 '__version__': '1.0',
 'annotations': array([[(array([[39]], dtype=uint8), array([[116]], dtype=uint8), array([[569]], dtype=uint16), array([[375]], dtype=uint16), array([[14]], dtype=uint8), array(['00001.jpg'], dtype='<U9')),
         (array([[36]], dtype=uint8), array([[116]], dtype=uint8), array([[868]], dtype=uint16), array([[587]], dtype=uint16), array([[3]], dtype=uint8), array(['00002.jpg'], dtype='<U9')),
         (array([[85]], dtype=uint8), array([[109]], dtype=uint8), array([[601]], dtype=uint16), array([[381]], dtype=uint16), array([[91]], dtype=uint8), array(['00003.jpg'], dtype='<U9')),
         ...,
         (array([[26]], dtype=uint8), array([[246]], dtype=uint8), array([[660]], dtype=uint16), array([[449]], dtype=uint16), array([[163]], dtype=uint8), array(['08142.jpg'], dtype='<U9')),
         (array([[78]], dtype=uint8), array([[526]], dtype=uint16), 

In [57]:
# "ann" is a structured array. It represents 
# the whole annotation table and is wrapped up 
# by a numpy array. That's why its shape is (1, 8144)
ann = x["annotations"]
ann.shape

(1, 8144)

In [58]:
# This represents the annotations without wrapping
ann[0]

array([(array([[39]], dtype=uint8), array([[116]], dtype=uint8), array([[569]], dtype=uint16), array([[375]], dtype=uint16), array([[14]], dtype=uint8), array(['00001.jpg'], dtype='<U9')),
       (array([[36]], dtype=uint8), array([[116]], dtype=uint8), array([[868]], dtype=uint16), array([[587]], dtype=uint16), array([[3]], dtype=uint8), array(['00002.jpg'], dtype='<U9')),
       (array([[85]], dtype=uint8), array([[109]], dtype=uint8), array([[601]], dtype=uint16), array([[381]], dtype=uint16), array([[91]], dtype=uint8), array(['00003.jpg'], dtype='<U9')),
       ...,
       (array([[26]], dtype=uint8), array([[246]], dtype=uint8), array([[660]], dtype=uint16), array([[449]], dtype=uint16), array([[163]], dtype=uint8), array(['08142.jpg'], dtype='<U9')),
       (array([[78]], dtype=uint8), array([[526]], dtype=uint16), array([[1489]], dtype=uint16), array([[908]], dtype=uint16), array([[112]], dtype=uint8), array(['08143.jpg'], dtype='<U9')),
       (array([[20]], dtype=uint8), arra

In [88]:
df = pd.DataFrame(ann[0])
df.head()

,bbox_x1,bbox_y1,bbox_x2,bbox_y2,class,fname
0,[[39]],[[116]],[[569]],[[375]],[[14]],[00001.jpg]
1,[[36]],[[116]],[[868]],[[587]],[[3]],[00002.jpg]
2,[[85]],[[109]],[[601]],[[381]],[[91]],[00003.jpg]
3,[[621]],[[393]],[[1484]],[[1096]],[[134]],[00004.jpg]
4,[[14]],[[36]],[[133]],[[99]],[[106]],[00005.jpg]


# Process the data

In [94]:
def process_anno(file_path, save_path=None):
    mat = loadmat(file_path)
    df = pd.DataFrame(mat["annotations"][0])
    cols = df.columns
    
    for col in cols:
        df[col] = df[col].apply(np.reshape, newshape=())
    if save_path is not None:
        df.to_csv(save_path, index=False)
    return df

In [98]:
df_train = process_anno(file_path="cars_train_annos.mat", save_path="cars_train_annos.csv")
df_train.head()

,bbox_x1,bbox_y1,bbox_x2,bbox_y2,class,fname
0,39,116,569,375,14,00001.jpg
1,36,116,868,587,3,00002.jpg
2,85,109,601,381,91,00003.jpg
3,621,393,1484,1096,134,00004.jpg
4,14,36,133,99,106,00005.jpg


In [101]:
df_test = process_anno(file_path="cars_test_annos.mat", save_path="cars_test_annos.csv")
df_test.head()

,bbox_x1,bbox_y1,bbox_x2,bbox_y2,fname
0,30,52,246,147,00001.jpg
1,100,19,576,203,00002.jpg
2,51,105,968,659,00003.jpg
3,67,84,581,407,00004.jpg
4,140,151,593,339,00005.jpg
